# HEDNO Load Data and Factor IDs

In [ ]:
import pandas as pd

from hedno.pipelines.preprocess import nodes as prp

In [ ]:
pd.options.display.memory_usage = "deep"
pd.options.display.show_dimensions = True

## Categories

In [ ]:
prp.PHASE_TYPE

In [ ]:
prp.REQUEST_TYPE

In [ ]:
prp.VOLTAGE_TYPE

## Load utility datasets

### Rates

In [ ]:
preprocessed_rates = (
    catalog
    .load("raw_rates")
    .pipe(prp.preprocess_rates)
)

preprocessed_rates

In [ ]:
preprocessed_rates.info(show_counts=True)

### Powers

In [ ]:
preprocessed_powers = (
    catalog
    .load("raw_powers")
    .pipe(prp.preprocess_powers)
)

preprocessed_powers

In [ ]:
preprocessed_powers.info(show_counts=True)

### Suppliers

In [ ]:
preprocessed_suppliers = (
    catalog
    .load("raw_suppliers")
    .pipe(prp.preprocess_suppliers)
)

preprocessed_suppliers

In [ ]:
preprocessed_suppliers.info(show_counts=True)

### Uses

In [ ]:
preprocessed_uses = (
    catalog
    .load("raw_uses")
    .pipe(prp.preprocess_uses)
)

preprocessed_uses

In [ ]:
preprocessed_uses.info(show_counts=True)

## Load main datasets

### Records

#### Test

In [ ]:
records_test = catalog.load("raw_records_test")

records_test

In [ ]:
records_test.info(show_counts=True)

#### Train

In [ ]:
records_train = catalog.load("raw_records_train")

records_train

In [ ]:
records_train.info(show_counts=True)

#### Merge and preprocess

In [ ]:
records = (
    prp
    .concat_annotated_train_and_test(
        test_df=records_test,
        train_df=records_train
    )
    .pipe(prp.preprocess_records)
)

records

In [ ]:
records.info(show_counts=True)

#### Check Unique Key

In [ ]:
RECORDS_UNIQUE_KEY = ["ACCT_NBR", "successor"]

records[records.duplicated(subset=RECORDS_UNIQUE_KEY, keep=False)]

`(ACCT_NBR, successor)` is UNIQUE KEY in the merged `Records` dataset. Use the accompanying `test` entry in `Records` as source of truth for train VS test labelling, and **ignore** any such train VS test distinction in `Consumptions`, `Representations` and `Requests`.

### Representations

#### Test

In [ ]:
representations_test = catalog.load("raw_representations_test")

representations_test

In [ ]:
representations_test.info(show_counts=True)

#### Train

In [ ]:
representations_train = (
    catalog
    .load("raw_representations_train")
    .pipe(prp.hotfix_representations_train)
)

representations_train

In [ ]:
representations_train.info(show_counts=True)

#### Merge and preprocess

In [ ]:
representations = (
    prp
    .concat_train_and_test(
        test_df=representations_test,
        train_df=representations_train
    )
    .pipe(prp.preprocess_representations)
)

representations

In [ ]:
representations.info(show_counts=True)

### Requests

#### Test

In [ ]:
requests_test = catalog.load("raw_requests_test")

requests_test

In [ ]:
requests_test.info(show_counts=True)

#### Train

In [ ]:
requests_train = (
    catalog
    .load("raw_requests_train")
    .pipe(prp.hotfix_requests_train)
)

requests_train

In [ ]:
requests_train.info(show_counts=True)

#### Merge and preprocess

In [ ]:
requests = (
    prp
    .concat_train_and_test(
        test_df=requests_test,
        train_df=requests_train
    )
    .pipe(prp.preprocess_requests)
)

requests

In [ ]:
requests.info(show_counts=True)

### Consumptions

#### Test

In [ ]:
constt = catalog.load("raw_consumptions_test")

constt

In [ ]:
constt.info(show_counts=True)

#### Train

In [ ]:
constr = catalog.load("raw_consumptions_train")

constr

In [ ]:
constr.info(show_counts=True)

#### Merge and preprocess

In [ ]:
consumptions = (
    prp
    .concat_train_and_test(
        test_df=constt,
        train_df=constr
    )
    .pipe(prp.preprocess_consumptions)
)

consumptions

In [ ]:
consumptions.info(show_counts=True)

### PowerThefts

In [ ]:
powerthefts = (
    catalog
    .load("raw_powerthefts")
    .pipe(prp.hotfix_powerthefts)
    .pipe(prp.preprocess_powerthefts)
)

powerthefts

In [ ]:
powerthefts.info(show_counts=True)

### Tests

In [ ]:
preprocessed_tests = (
    catalog
    .load("raw_tests")
    .pipe(prp.preprocess_tests)
)

preprocessed_tests

In [ ]:
preprocessed_tests.info(show_counts=True)

## Construct ID spaces

### Acounts

In [ ]:
%%time

accounts = prp.collect_accounts(
    consumptions=consumptions,
    powerthefts=powerthefts,
    records=records,
    representations=representations,
    requests=requests,
    tests=preprocessed_tests
)

accounts

In [ ]:
accounts.info(show_counts=True)

### Meters

In [ ]:
%%time

meters = prp.collect_meters(consumptions)

meters

In [ ]:
meters.info(show_counts=True)

### Methods

In [ ]:
methods = prp.collect_methods(powerthefts=powerthefts)

methods

In [ ]:
methods.info(show_counts=True)

### Places

In [ ]:
places = prp.collect_places(powerthefts=powerthefts)

places

In [ ]:
places.info(show_counts=True)

### Powers

In [ ]:
powers = prp.collect_powers(powers=preprocessed_powers, records=records)

powers

In [ ]:
powers.info(show_counts=True)

### Suppliers

In [ ]:
suppliers = prp.collect_suppliers(
    representations=representations,
    suppliers=preprocessed_suppliers
)

suppliers

In [ ]:
suppliers.info(show_counts=True)

#### Missing suppliers

The following supplier(s) appear in `Representations` but not in `Suppliers`, thus their `supplier_name` is missing.

Note: Supplier `Π` also appears in `Representations` only, but has been hotfixed to (known supplier) `π`, as there was only 1 such entry.

In [ ]:
suppliers.query("supplier_name.isna()")

### Rates

In [ ]:
rates = prp.collect_rates(consumptions=consumptions, rates=preprocessed_rates)

rates

In [ ]:
rates.info(show_counts=True)

#### Missing rates
The following rates appear in `Consumptions` but not in `Rates`, and thus their `rate_name` is missing.

In [ ]:
rates.query("rate_name.isna()")

### Suppliers

In [ ]:
suppliers = prp.collect_suppliers(
    representations=representations,
    suppliers=preprocessed_suppliers
)

suppliers

In [ ]:
suppliers.info(show_counts=True)

#### Missing suppliers

The following supplier(s) appear in `Representations` but not in `Suppliers`, thus their `supplier_name` is missing.

Note: Supplier `Π` also appears in `Representations` only, but has been hotfixed to (known supplier) `π`, as there was only 1 such entry.

In [ ]:
suppliers.query("supplier_name.isna()")

### Uses

In [ ]:
uses = prp.collect_uses(records=records, uses=preprocessed_uses)

uses

In [ ]:
uses.info(show_counts=True)

## Factor datasets with respect to ID spaces

### Consumptions

In [ ]:
%%time

factored_consumptions = prp.factor_consumptions(
    consumptions=consumptions,
    accounts=accounts,
    meters=meters,
    rates=rates
)

factored_consumptions

In [ ]:
factored_consumptions.info(show_counts=True)

### PowerThefts

In [ ]:
factored_powerthefts = prp.factor_powerthefts(
    powerthefts=powerthefts,
    accounts=accounts,
    methods=methods,
    places=places
)

factored_powerthefts

In [ ]:
factored_powerthefts.info(show_counts=True)

### Locations and Records

In [ ]:
factored_records_and_locations = prp.factor_records_and_locations(
    records=records,
    accounts=accounts,
    powers=powers,
    uses=uses
)

factored_records_and_locations

In [ ]:
factored_records_and_locations.info(show_counts=True)

#### Locations
Location is only supposed to change based on `account_id`.

In [ ]:
factored_locations = prp.factor_locations(
    factored_records_and_locations=factored_records_and_locations
)

factored_locations

In [ ]:
factored_locations.info(show_counts=True)

##### Check Unique Keys

In [ ]:
LOCATION_UNIQUE_KEY = ["account_id"]

factored_locations[
    factored_locations.duplicated(subset=LOCATION_UNIQUE_KEY, keep=False)
]

`account_id` is indeed UNIQUE KEY to `x` and `y`.

#### Records

In [ ]:
factored_records = prp.factor_records(
    factored_records_and_locations=factored_records_and_locations
)

factored_records

In [ ]:
factored_records.info(show_counts=True)

### Representations

In [ ]:
factored_representations = prp.factor_representations(
    representations=representations,
    accounts=accounts,
    suppliers=suppliers
)

factored_representations

In [ ]:
factored_representations.info(show_counts=True)

### Requests

In [ ]:
factored_requests = prp.factor_requests(requests=requests, accounts=accounts)

factored_requests

In [ ]:
factored_requests.info(show_counts=True)

### Tests

In [ ]:
tests = prp.factor_tests(tests=preprocessed_tests, accounts=accounts)

tests

In [ ]:
tests.info(show_counts=True)

## Save factored datasets

In [ ]:
%%time

catalog.save("factored_consumptions", factored_consumptions)
catalog.save("factored_locations", factored_locations)
catalog.save("factored_powerthefts", factored_powerthefts)
catalog.save("factored_records", factored_records)
catalog.save("factored_representations", factored_representations)
catalog.save("factored_requests", factored_requests)

## Save primary datasets

In [ ]:
%%time

catalog.save("accounts", accounts)
catalog.save("meters", meters)
catalog.save("methods", methods)
catalog.save("places", places)
catalog.save("powers", powers)
catalog.save("rates", rates)
catalog.save("suppliers", suppliers)
catalog.save("tests", tests)
catalog.save("uses", uses)